In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import concurrent
from time import sleep
from IPython.display import display, Math, Latex

O que este notebook ira realizar:
-------------------------------

1. Acessar o site da olx utilizando a biblioteca **BeautifulSoup**, ecnontrar a lista de preços e salvar cada item em uma lista de "Tags"
2. Encontrar os valores (municipio, estado , categoria, titulo, preco, link) de cada anuncio na lista
3. Salvar em um DataFrame do Pandas
4. Acessar cada um dos links de cada anuncio no DataFrame, no campo **href** e retira dele o CEP de cada anuncio criando um dicionario neste formato {"href":"cep"} **(Esta parte do codigo precisa ser aprimorada atualmente demora em torno de 5 à 7min para executar)**
5. Adicionar o dicionario de CEP ao dataframe de anuncios utilizando a função _.map_ do Pandas 
6. Salvar o DataFrame resultante em uma planilha Excel

In [2]:
labels = ['local', 'categoria', 'titulo', 'preco', 'href']
cep_dict = {}
url = "http://sp.olx.com.br/sao-paulo-e-regiao/zona-leste/imoveis?o="

In [3]:
def retornaSoupOLX(url, num):
    li_list = []
    for iter in range(num):
        r = requests.get(str(url)+str(iter))
        soup = BeautifulSoup(r.content, 'html.parser')
        for ultag in soup.find_all('ul', {'class': 'list', 'id':'main-ad-list' }):
            for litag in ultag.find_all(lambda tag: tag.name == 'li' and tag.get('class') == ['item']):
                li_list.append(litag)
    return li_list

In [4]:
def tiraEspecialString(string):
    return string.text.replace('\n','').replace('\t','').replace('R$ ', '').replace('.', '')

In [5]:
def tiraEspecialFloat(string):
    return string.text.replace('\n','').replace('\t','').replace('R$ ', '').replace('.', '').replace(',', '.')

In [6]:
%%time

def criaListaValores(li_list):
    valores = []
    for elements in li_list:
        x = BeautifulSoup(str(elements), 'html.parser')
        #encontra a parte onde estão as informaçoes no html
        info = x.find('div', {'class':'col-2'})
        #encontra a string onde estão o municipio e estado separados por virgula
        cidade_estado = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-2'}).find('p', {'class':'text detail-region'}))
        #encontra a categoria do anuncio
        categoria = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-2'}).find('p', {'class':'text detail-category'}))
        #encontra o titulo do anuncio
        titulo = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-1 mb5px'}).find('h3', {'class':'OLXad-list-title mb5px'}))
        #encontra o html do preço
        preco = x.find('div', {'class':'col-3'})
        
        #trata o html do preço verificando se existe ou não
        if preco.find('p', {'class':'OLXad-list-price'}):
            preco = tiraEspecialFloat(x.find('div', {'class':'col-3'}).find('p', {'class':'OLXad-list-price'}))
        else:
            preco = 0.0
        #retorna a tag href "link do anuncio"
        href=x.find('a', {'class':'OLXad-list-link'})
        
        #retorna o conteudo do atributo href na tag href
        href = href['href']
        
        valores.append([cidade_estado, categoria, titulo, preco, href])
    return valores
        

Wall time: 0 ns


In [7]:
def findCEP(url):
    regx = re.compile('\d{5}-\d{3}')
    r = requests.get(url)
    sleep(2)
    soup = BeautifulSoup(r.content, 'html.parser')
    cep = soup.find(lambda tag: tag.name == 'strong' and tag.get('class') == ['description'] and regx.match(tag.text.strip(' \t\n\r')) is not None)
    if cep:
        cep_dict.update({url:cep.text.strip(' \t\n\r')})
    else:
        cep_dict.update({url:''})

In [8]:
%%time
soup = retornaSoupOLX(url, 10)

Wall time: 20.5 s


In [9]:
%%time
valores = criaListaValores(soup)

Wall time: 3.96 s


In [10]:
%%time
df = pd.DataFrame(valores, columns=labels)

Wall time: 3 ms


In [11]:
list_href = list(df['href'])

In [12]:
%%time
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as e:
     e.map(findCEP, list_href)

Wall time: 1min 37s


In [13]:
df['cep'] = df['href'].map(cep_dict)

In [14]:
df = pd.concat([df , df['local'].str.split(',', n=1, expand=True)], axis =1)

In [15]:
# df.to_excel('OLX_Scrap1.xlsx')

In [16]:
df.drop(['local', 0], axis=1, inplace=True)

In [17]:
df.columns = ['categoria', 'titulo', 'preco', 'href', 'cep', 'bairro']

In [18]:
df['categoria'] = df['categoria'].str.replace('- Anúncio Profissional', 'Profissional')
df['categoria'] = df['categoria'].str.replace('Profissional', '- Profissional')
df['categoria'], df['profissional'] = df['categoria'].str.split('-', 1).str

In [19]:
df.categoria.unique()

array(['Casas', 'Indústria e comércio', 'Apartamentos',
       'Terrenos, sítios e fazendas', 'Aluguel de quartos', 'Temporada'], dtype=object)

In [20]:
df['profissional']=df.profissional.replace(' Profissional', 'S')
df['profissional']=df.profissional.fillna('N')
df.head()

,categoria,titulo,preco,href,cep,bairro,profissional
0,Casas,Sobrado na Vila Granada,560000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,03622-000,Vila Granada,N
1,Indústria e comércio,Trailer Móvel + Ponto Comercial,29000,http://sp.olx.com.br/sao-paulo-e-regiao/indust...,08280-190,Cidade Líder,N
2,Apartamentos,Lindo Apartamento próximo ao Shopping Anália F...,615000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,03348-040,Vila Invernada,S
3,Apartamentos,Apto sem condomínio,100000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,08320-390,Parque São Rafael,N
4,Casas,Casa,400000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,08070-310,Parque Cruzeiro do Sul,S


In [30]:
df['bairro'] = df['bairro'].str.replace(' ','', n=1)

In [31]:
df.loc[:,df.columns != 'href'].to_excel('../1-OLX_WebScrap/1_OLX_SEM_CEP.xlsx')
df.to_excel('../1-OLX_WebScrap/1_OLX_COM_CEP.xlsx')